#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/

## Задачи для совместного разбора

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [ ]:
import torch

In [ ]:
class Neuron:
  def __init__(self, weights, bias):
    self.w = weights
    self.b = bias
    pass

  def forward(self, inputs):
    y_pred = inputs.dot(self.w) + self.b
    return y_pred


In [ ]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = 3.14

In [ ]:
n = Neuron(weights, bias)
res = n.forward(inputs)
print(res)

tensor(4.8400)


2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
class ReLU:
  def forward(self, inputs):

    return torch.maximum(inputs, torch.tensor(0.0))


In [ ]:
inputs = torch.randn(4, 3)

relu = ReLU()
output = relu.forward(inputs)

print(inputs)
print(output)

tensor([[ 2.5357,  0.6343,  0.7144],
        [ 1.3253,  0.1305,  0.0154],
        [-0.2082, -0.1816,  1.2887],
        [-1.8082,  0.1260, -1.7094]])
tensor([[2.5357, 0.6343, 0.7144],
        [1.3253, 0.1305, 0.0154],
        [0.0000, 0.0000, 1.2887],
        [0.0000, 0.1260, 0.0000]])


3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [ ]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return ((y_true - y_pred)**2).mean()

In [ ]:
y_pred = torch.tensor([1.0, 3.0, 5.0])
y_true = torch.tensor([2.0, 3.0, 4.0])

mse = MSELoss()

output = mse.forward(y_pred, y_true)

output

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1_1"></p>

1\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [ ]:
class Linear:
    def __init__(self, n_neurons, n_features, bias: bool = False):

      self.w = torch.randn(n_features, n_neurons)
      self.b = torch.randn(1, n_neurons)

    def forward(self, inputs):

        y_pred = torch.mm(inputs, self.w) + self.b
        return y_pred

In [ ]:
# inputs: batch_size(3) x 4
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [ ]:
l = Linear(7, inputs.shape[1])

res = l.forward(inputs)
print(res, res.numpy().shape==(inputs.shape[0], l.w.shape[1]))


tensor([[ 3.2226,  1.9927,  4.9358, -0.7066,  4.6427,  3.6637,  3.9594],
        [-9.3001,  1.2318,  1.9951, -1.0720,  4.2394, -9.0200,  3.6165],
        [ 5.4695,  2.3114,  4.6244, -3.3402,  8.4091, -0.2947, -2.8384]]) True


<p class="task" id="1_2"></p>

1\.2 Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

In [ ]:
class Linear:
    def __init__(self, n_neurons, n_features, bias: bool = False):

      self.w = torch.randn(n_features, n_neurons)
      self.b = torch.randn(1, n_neurons) if bias else None

    def forward(self, inputs):

        y_pred = torch.mm(inputs, self.w)
        if self.b is not None:
          y_pred+=self.b
        return y_pred


inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

l1 = Linear(n_neurons=13, n_features=inputs.shape[1], bias=True)
l2 = Linear(n_neurons=7, n_features = 13, bias=True)

hidden_output = l1.forward(inputs)
output = l2.forward(hidden_output)

print(f"Предсказания первого слоя:\n{hidden_output}")
print(f"Предсказания второго слоя:\n{output}  {output.shape}")



Предсказания первого слоя:
tensor([[ -1.8713,  -5.2386,  -0.6393,  -3.1091,   1.6184,   5.8310,   3.7517,
          -3.7118,  -1.0760,  -4.3951,  -4.3477,   2.3365,  -7.7170],
        [  5.0663,  -5.5126,   2.9190, -11.5274,  -0.9474,   0.3977,   1.5288,
          -8.2771,   0.2199,  -9.0566,   3.5872,  -0.2932,  -4.9648],
        [ -7.1866,  -3.8672,  -3.2966,   0.3315,   0.3258,   5.3511,   5.6888,
          -3.5277,  -2.0159,   0.9868,  -9.3009,   2.8933,  -3.1581]])
Предсказания второго слоя:
tensor([[ -5.3807, -29.8385,   6.1046,  -4.9459,   0.3012, -25.2303,  15.1119],
        [ 11.0070, -37.6689,   2.7706, -15.8890, -29.4547, -20.1983,  21.7935],
        [-24.5673, -14.9141,   2.1156,   0.8492,  12.9448, -16.4065,   2.1643]])  torch.Size([3, 7])


### Создание функций активации

<p class="task" id="2_1"></p>

2\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

In [ ]:
class Softmax:
    def forward(self, inputs):
        exp_items = torch.exp(inputs - torch.max(inputs, dim=1, keepdim=True).values)
        return exp_items / torch.sum(exp_items, dim=1, keepdim=True)



inputs = torch.randn(4, 3)

sfm = Softmax()
output = sfm.forward(inputs)

print(output)
print(output.sum(dim=1))

tensor([[0.1362, 0.1311, 0.7328],
        [0.0868, 0.6847, 0.2285],
        [0.6546, 0.2526, 0.0929],
        [0.7685, 0.1539, 0.0776]])
tensor([1., 1., 1., 1.])


<p class="task" id="2_2"></p>

2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
class ELU:
    def __init__(self, alpha: float=1.0):

        self.alpha = alpha

    def forward(self, inputs):

        return torch.where(inputs >= 0, inputs, self.alpha * (torch.exp(inputs) - 1))


inputs = torch.randn(4, 3)

elu = ELU(alpha=1.0)

print(inputs)
print("\n")
print(elu.forward(inputs))

tensor([[-0.8611, -1.2210,  0.2200],
        [ 0.1060, -0.4769, -0.0339],
        [ 0.6881, -0.9139,  0.1669],
        [-0.9550, -0.6524, -0.1858]])


tensor([[-0.5773, -0.7051,  0.2200],
        [ 0.1060, -0.3793, -0.0333],
        [ 0.6881, -0.5990,  0.1669],
        [-0.6152, -0.4792, -0.1695]])


### Создание функции потерь

<p class="task" id="3_1"></p>

3\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$
<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

In [ ]:
class CrossEntropyLoss:
  def forward(self, y_pred, y_true):

    N = y_pred.shape[0]
    y_pred = torch.clamp(y_pred, min=1e-9, max=1.0)
    log_prob = torch.log(y_pred)
    loss = -log_prob[range(N), y_true].mean()
    return loss

inputs = torch.tensor([[1, 2, 3, 2.5],
                        [2, 5, -1, 2],
                        [-1.5, 2.7, 3.3, -0.8]], dtype=torch.float32)

l = torch.nn.Linear(4, 3)
output = l(inputs)

sfm = torch.nn.Softmax()
pred = sfm(output)

y = torch.tensor([1, 0, 0], dtype=torch.long)

loss_f = CrossEntropyLoss()
loose = loss_f.forward(pred, y)

print(loose.item())





1.586622714996338


In [ ]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                        [2, 5, -1, 2],
                        [-1.5, 2.7, 3.3, -0.8]])
y = torch.tensor([1, 0, 0])

<p class="task" id="3_2"></p>

3.2 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

In [ ]:
class MSERegularized:
    def __init__(self, lambda_):
        self.l = lambda_

    def data_loss(self, y_pred, y_true):

        return torch.mean((y_true - y_pred)**2)


    def reg_loss(self, weights):

        return self.l * torch.sum(weights**2)


    def forward(self, y_pred, y_true, weights):
        return self.data_loss(y_pred, y_true) + self.reg_loss(weights)


y_pred = torch.tensor([-0.5, 1, 1.7])
y_true = torch.tensor([0, 0.6, 2.3])
weights = torch.normal(0, 5, (10, 1))

lambda_ = 0.1

MSEr = MSERegularized(lambda_)
loss = MSEr.forward(y_pred, y_true, weights)
print(loss.item())

27.231433868408203


In [ ]:
y_pred = torch.tensor([-0.5, 1, 1.7])
y_true = torch.tensor([0, 0.6, 2.3])
weights = torch.normal(0, 5, (10, 1))

## Обратная связь
- [ ] Хочу получить обратную связь по решению